In [1]:
import pandas as pd
from pathlib import Path

# Path to your dataset
DATA_PATH = Path("retail_dynamic_pricing_dataset.csv")  # or full path if needed

df = pd.read_csv(DATA_PATH)
print(df.head())
print(df.shape)



         date product_id     category  season  cost_price  competitor_price  \
0  2024-01-01      P0001    Household  Spring       47.65             17.13   
1  2024-01-02      P0001  Electronics  Winter       24.07             92.02   
2  2024-01-03      P0001       Snacks  Spring       24.86             46.96   
3  2024-01-04      P0001  Electronics  Summer       40.33            107.27   
4  2024-01-05      P0001  Electronics  Autumn       12.04             53.40   

   stock_level  promotion day_of_week  current_price  discount  \
0          162          0         Mon          80.56       0.0   
1          437          0         Fri          29.29       0.0   
2          584          0         Sat          43.60       0.0   
3          878          0         Sat          57.40       0.0   
4          921          0         Mon          16.11       0.0   

   effective_price  units_sold  revenue  
0            80.56           1    80.56  
1            29.29          50  1464.50  
2 

In [2]:
# Create extra features
df["price_diff_vs_competitor"] = df["effective_price"] - df["competitor_price"]
df["margin"] = df["effective_price"] - df["cost_price"]

feature_cols = [
    "effective_price",
    "competitor_price",
    "cost_price",
    "stock_level",
    "promotion",
    "category",
    "season",
    "day_of_week",
    "price_diff_vs_competitor",
    "margin",
]

X = df[feature_cols]
y = df["units_sold"]


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

numeric_features = [
    "effective_price",
    "competitor_price",
    "cost_price",
    "stock_level",
    "promotion",
    "price_diff_vs_competitor",
    "margin",
]

categorical_features = ["category", "season", "day_of_week"]

numeric_transformer = "passthrough"
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Model
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

# Pipeline = preprocessing + model
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", rf),
])

# Train
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R²:  {r2:.3f}")


MAE: 21.22
R²:  0.622


In [5]:
import joblib
Path("models").mkdir(exist_ok=True)

joblib.dump(model, "models/units_sold_model.pkl")
print("Model saved to models/units_sold_model.pkl")


Model saved to models/units_sold_model.pkl


In [6]:
import numpy as np

def suggest_optimal_price_for_row(row, model, feature_cols, 
                                  price_min_factor=0.8, price_max_factor=1.2, n_steps=20):
    """
    row: pd.Series representing one product-day context
    model: trained sklearn Pipeline
    feature_cols: list of feature column names used for training
    """
    current_price = row["effective_price"]
    competitor_price = row["competitor_price"]
    cost_price = row["cost_price"]

    # Generate candidate prices around current price
    min_price = current_price * price_min_factor
    max_price = current_price * price_max_factor
    candidate_prices = np.linspace(min_price, max_price, n_steps)

    best_price = None
    best_units = None
    best_revenue = -np.inf

    for p in candidate_prices:
        temp = row.copy()
        temp["effective_price"] = p
        temp["price_diff_vs_competitor"] = p - competitor_price
        temp["margin"] = p - cost_price

        temp_df = pd.DataFrame([temp[feature_cols]])

        pred_units = model.predict(temp_df)[0]
        pred_units = max(pred_units, 0)  # no negative sales

        revenue = p * pred_units

        if revenue > best_revenue:
            best_revenue = revenue
            best_price = p
            best_units = pred_units

    return best_price, best_units, best_revenue


In [7]:
# Pick one product's latest row
pid = "P0001"
row = df[df["product_id"] == pid].iloc[-1]

opt_price, opt_units, opt_rev = suggest_optimal_price_for_row(row, model, feature_cols)

current_price = row["effective_price"]
current_units = row["units_sold"]
current_rev = current_price * current_units

print("Product:", pid)
print(f"Current price: {current_price:.2f}, units: {current_units}, revenue: {current_rev:.2f}")
print(f"Suggested price: {opt_price:.2f}, predicted units: {opt_units:.1f}, predicted revenue: {opt_rev:.2f}")
print(f"Revenue gain: {opt_rev - current_rev:.2f}")


Product: P0001
Current price: 29.68, units: 32, revenue: 949.76
Suggested price: 29.99, predicted units: 75.5, predicted revenue: 2263.98
Revenue gain: 1314.22
